# Exercise 4

(c) 2019 Justin Bois. With the exception of pasted graphics, where the source is noted, this work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT).

This document was prepared at [Caltech](http://www.caltech.edu) with financial support from the [Donna and Benjamin M. Rosen Bioengineering Center](http://rosen.caltech.edu).

<img src="caltech_rosen.png">

*This exercise was generated from a Jupyter notebook.  You can download the notebook [here](e04.ipynb).*

<hr />

In [1]:
import numpy as np

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

<hr />

## Exercise 4.0: Complete practice

If you have not already, complete the exercises from [lesson 29](l29_practice_with_numpy.html) and [lessons 31 and 32](l31_32_practice_with_hackerstats.html).

<br />

## Exercise 4.1: Data collapse

Rob Phillips wrote a review paper a couple of years ago that I enjoyed entitled ["Napoleon is in Equilibrium."](https://dx.doi.org/10.1146%2Fannurev-conmatphys-031214-014558). In the paper, he demonstrated that when you plot data in a certain way, they demonstrate **data collapse**. The idea here is that if you choose the right thing to plot on the $x$ and $y$ axes, data from a variety of sources collapse onto a single universal curve. In this exercise, you will hone your NumPy and Bokeh skills in making plots exhibiting data collapse. 

This analysis comes from Rob's paper, and the data come from [Daber, Sochor, and Lewis, *J. Mol. Biol.*, **409**, 76–87, 2011](https://doi.org/10.1016/j.jmb.2011.03.057). The authors were studying how different mutants of the lac repressor affect gene expression. They hooked the lac promoter up to a fluorescent protein reporter. They made a mutant with no lac repressor to get a measurement of the gene expression level (quantified by the fluorescent signal) in the absence of repressor. Then then looked at how the presence of a repressor served to decrease the expression level of the lac gene. The ratio of the repressed fluorescence to the totally unrepressed fluorescence is the **fold change** in repression. They can block repression by adding IPTG, which binds the lac repressor, rendering it ineffective at repressing gene expression (so IPTG is called an "inducer," since it turns on gene expression). So, for a given experiment, the authors measured fold change as a function of IPTG concentration. They measured the fold change for wild type, plus two mutants, Q18M and Q18A.

We will not derive it here (it comes from a generalization of the [Monod-Wyman-Changeux model](https://en.wikipedia.org/wiki/Monod-Wyman-Changeux_model)), but the theoretical expression for the fold change as a function of IPTG concentration, $c$, is

\begin{align}
\text{fold change} = \left[1 + \frac{\frac{R}{K}\left(1 + c/K_\mathrm{d}^\mathrm{A}\right)^2}{\left(1 + c/K_\mathrm{d}^\mathrm{A}\right)^2 + K_\mathrm{switch}\left(1 + c/K_\mathrm{d}^\mathrm{I}\right)^2}\right]^{-1}.
\end{align}

The parameters are:

|Parameter|Description|Value|Units|
|:--:|:--:|:--:|:--:|
|$K_\mathrm{d}^\mathrm{A}$|dissoc. const. for active repressor binding IPTG|0.017|mM$^{-1}$|
|$K_\mathrm{d}^\mathrm{I}$|dissoc. const. for inactive repressor binding IPTG|0.002|mM$^{-1}$|
|$K_\mathrm{switch}$|equil. const. for switching active/inactive|5.8|---|
|$K$|dissoc. const. for active repressor binding operator|?|mM$^{-1}$|
|$R$|number of repressors in cell|?|---|

The values of $K_\mathrm{d}^\mathrm{A}$, $K_\mathrm{d}^\mathrm{I}$, and $K_\mathrm{switch}$ were measured in the Daber, Sochor, and Lewis paper, and, as I mentioned before, are the same for all mutants. You can see in the expression for the fold change that $R$ and $K$ always appear as a ratio, $R/K$, so we can only determine this *ratio*, $R/K$, for each mutant. They are, for the respective mutants:

|Mutant|$R/K$|
|:--:|:--:|
|WT|141.5 mM$^{-1}$|
|Q18A|16.56 mM$^{-1}$|
|Q18M|1332 mM$^{-1}$|

Now let's get started with the analysis.

**a)** Load in the three data sets. They are in the files `~/git/data/wt_lac.csv`, `~/git/data/q18m_lac.csv`, and `~/git/data/q18a_lac.csv`. You should put them in a single `DataFrame` with an added column for genotype. This can be accomplished, for example, with `pd.concat()`.

**b)** Make a plot of fold change vs. IPTG concentration for each of the three mutants. Think: should any of the axes have a logarithmic scale?

**c)** Write a function with the signature `fold_change(c, RK, KdA=0.017, KdI=0.002, Kswitch=5.8)` to compute the theoretical fold change. It should allow `c`, the concentration of IPTG, to be passed in as a NumPy array or scalar, and `RK`, the $R/K$ ratio, must be a scalar. Remember, with NumPy arrays, you don't have to write **`for`** loops to do operations to each element of the array.

**d)** You will now plot a smooth curve showing the theoretical fold change for each mutant. 
>1. Make an array of closely spaced points for the IPTG concentration. *Hint*: The function `np.logspace()` will be useful.
2. Compute the theoretical fold change based on the given parameters using the function you wrote in part (c).
3. Plot the smooth curves on the same plot with the data.

**e)** If we look at the functional form of the fold change and at the parameters we are given, we see that only $R/K$ varies from mutant to mutant. I told you this *a priori*, but we didn't really know it. Daber, Sochol, and Lewis assumed that the binding to IPTG would be unaltered and the binding to DNA would be altered based on the position of the mutation in the lac repressor protein. Now, *if* this is true, then $R/K$ should be the only thing that varies. We can check this by seeing if the data collapse onto a single curve. To see how this works, we define the **Bohr parameter**, $F(c)$, as

\begin{align}
F(c) = -\ln\left(R/K\right) - \ln\left(\frac{\left(1 + c/K_\mathrm{d}^\mathrm{A}\right)^2}{\left(1 + c/K_\mathrm{d}^\mathrm{A}\right)^2 + K_\mathrm{switch}\left(1 + c/K_\mathrm{d}^\mathrm{I}\right)^2}\right).
\end{align}

The second term in the Bohr parameter is independent of the identity of the mutant, and the first term depends entirely upon it. Then, the fold change can be written as

\begin{align}
\text{fold change} = \frac{1}{1 + \mathrm{e}^{-F(c)}}.
\end{align}

So, if we make our $x$-axis be the Bohr parameter, all data should fall on the *same* curve. Hence the term, data collapse. (The Bohr parameter gets its name (as given by Rob Phillips) because it is inspired by the work of Christian Bohr (Niels's father), who discovered similar families of curves describing binding of oxygen to hemoglobin.)

Now, we will plot the theoretical curve of fold change versus Bohr parameter. 
>1. Write a function with call signature `bohr_parameter(c, RK, KdA=0.017, KdI=0.002, Kswitch=5.8)` that computes the Bohr parameter.
2. Write a function with call signature `fold_change_bohr(bohr_parameter)` that gives the fold change as a function of the Bohr parameter. 
3. Generate values of the Bohr parameter ranging from $-6$ to $6$ in order to make a smooth plot.
4. Compute the theoretical fold change as a function of the Bohr parameter and plot it as a gray line.


**f)** Now, for each experimental curve:
>1. Convert the IPTG concentration to a Bohr parameter using the given parameters.
2. Plot the experimental fold change versus the Bohr parameter you just calculated. Plot the data as dots on the same plot that you made the universal gray curve, making sure to appropriately annotate your plot.

Do you see data collapse? Does it make sense that only operator binding is changing from mutant to mutant? And importantly, the collapse demonstrates that all of the mutants are behaving according to the Monod-Wyman-Changeux model, and the mutations affect quantitative, not qualitative, changes in the behavior of the repressor.

<br />

## Exercise 4.2: Solving differential equations with NumPy

In addition to handling data, NumPy allows you do to *simulations*. For this exercise, we will use Euler's method to simulate a classic model for predator-prey population dynamics, the [Lotka-Volterra model](https://en.wikipedia.org/wiki/Lotka–Volterra_equations). As a warm-up example, we will simulate bacterial growth.

Bacterial growth can be modeled by the differential equation

\begin{align}
\frac{\mathrm{d}n}{\mathrm{d}t} = k n,
\end{align}

where $n$ is the number of bacteria and $k$ is the growth rate. The idea here is that the number of bacteria will grow faster the more bacteria we have, because there are more to divide. Analytically, we know the solution to this differential equation is

\begin{align}
n(t) = n_0 \mathrm{e}^{kt},
\end{align}

i.e., exponential growth. But suppose we did not know how to derive that. We could *simulate* the differential equation. We do this by discretizing time. Instead of a derivative, we have a change in $n$ over a change in time $t$.

\begin{align}
\frac{\mathrm{d}n}{\mathrm{d}t} \approx \frac{\Delta n}{\delta t} = k n.
\end{align}

Let's say we know $n$ and time zero, $n(0)$. Then $n$ at time $t = \Delta t$ is

\begin{align}
n(\Delta t) \approx n(0) + \Delta n = n(0) + k n(0).
\end{align}

More generally, we can write

\begin{align}
\frac{\mathrm{d}n}{\mathrm{d}t} = f(n),
\end{align}

and

\begin{align}
n(t+\Delta t) \approx n(t) + \Delta t\,f(n).
\end{align}

So, we can instruct Python to take our current value of $n$, and then add $\Delta t$ times $f(n)$ to get our new $n$ at a time just a bit later on, at $t + \Delta t$. We repeat this over and over again to move forward in time. Let's code that up!

In [2]:
# Specify parameter
k = 1

# Specify my little time step
delta_t = 0.01

# Make an array of time points, evenly spaced up to 10
t = np.arange(0, 10, delta_t)

# Make an array to store the number of bacteria
n = np.empty_like(t)

# Set the initial number of bacteria
n[0] = 100

# Write a for loop to keep updating n as time goes on
for i in range(1, len(t)):
    n[i] = n[i-1] + delta_t * k * n[i-1]

Ok! We just computed the time points and the number of bacteria, so we can just plot the result!

In [3]:
p = bokeh.plotting.figure(
    height=300,
    width=450,
    x_axis_label='time (units of 1/k)',
    y_axis_label='number of bacteria',
)

p.line(
    x=t,
    y=n,
    line_width=2
)

bokeh.io.show(p)

And there is the famous exponential growth!

This time stepping method is called [Euler's method](https://en.wikipedia.org/wiki/Euler_method), and what we're doing is called *numerical solution of a differential equation*.

**a)** Now it's time to simulate the Lotka-Volterra model. Since predator and prey both begin with "p," we'll call the predators foxes ($f$) and the prey rabbits ($r$). The differential equation describing the dynamics of the rabbit population is

\begin{align}
\frac{\mathrm{d}r}{\mathrm{d}t} = \alpha r - \beta f r.
\end{align}

The first term at the right hand side is exponential growth, the same you would expect for a growing bacterial colony. The second term is killing off due to predation. If $f$ is large, more rabbits get hunted down.

The differential equation describing the dynamics of the fox population is

\begin{align}
\frac{\mathrm{d}f}{\mathrm{d}t} = \delta f r - \gamma f.
\end{align}

The first term represents growth in the fox population by consumption of rabbits. The second term is the natural die-off of foxes.

Your task in this exercise is to numerically solve these two differential equations together and then plot the result. Use the following parameter values

```python
alpha = 1
beta = 0.2
delta = 0.3
gamma = 0.8
delta_t = 0.001
t = np.arange(0, 60, delta_t)
r[0] = 10
f[0] = 1
```

Even though there are now two differential equations, the procedure is the same, you update each by adding $\Delta t$ times the respective derivative.

When you plot the result, does it make sense?

**b)** [Bonus]  Euler's method is probably the simplest way to solve differential equations, and is by no means the best. SciPy has an ODE solver, `scipy.integrate.odeint()` that uses the more sophisticated and robust methods for solving systems of ODEs. Read the documentation about how `scipy.integrate.odeint()` works and use it to solve the Lotka-Volterra system of ODEs.

This problem is tough; I'm not giving you directions, and you are kind of on your own to read the documentation and figure it out. It may be useful to read [this tutorial](http://be150.caltech.edu/2019/handouts/02_intro_to_python_for_biological_circuits.html) I wrote to help students solve ODEs that come up in systems biology.

<br />

## Exercise 4.3: Hacker stats on bee sperm data

Neonicotinoid pesticides are thought to have inadvertent effects on service-providing insects such as bees. A study of this was [featured in the New York Times](http://www.nytimes.com/2016/07/29/science/neonicotinoid-insecticide-bee-sperm.html) in 2016. The original paper is [Straub, et al.,  Proc. Royal Soc. B 283(1835): 20160506](http://dx.doi.org/10.1098/rspb.2016.0506). Straub and coworkers put their [data in the Dryad repository](http://dx.doi.org/10.5061/dryad.bs515), which means we can work with it!

(Do you see a trend here? If you want people to think deeply about your results, explore them, learn from them, further science with them, *make your data publicly available.* Strongly encourage the members of your lab to do the same.)

We will look at the weight of drones (male bees) using the data set stored in `~/git/bootcamp/data/bee_weight.csv` and the sperm quality of drone bees using the data set stored in `~/git/bootcamp/data/bee_sperm.csv`.

**a)** Load the drone weight data in as a Pandas `DataFrame`.  Note that the unit of the weight is milligrams (mg).

**b)** Plot ECDFs of the drone weight for control and also for those exposed to pesticide. Do you think there is a clear difference?

**c)** Compute the mean drone weight for control and those exposed to pesticide. Compute 95% bootstrap confidence intervals on the mean.

**d)** Repeat parts (a)-(c) for drone sperm. Use the `'Quality'` column as your measure. This is defined as the percent of sperm that are alive in a 500 µL sample.

**e)** As you have seen in your analysis in part (d), both the control and pesticide treatments have some outliers with very low sperm quality. This can tug heavily on the mean. So, get 95% bootstrap confidence intervals for the *median* sperm quality of the two treatments.

<br />

## Exercise 4.4: Monte Carlo simulation of transcriptional pausing

In this exercise, we will put random number generation to use and do a **Monte Carlo simulation**. The term Monte Carlo simulation is a broad term describing techniques in which a large number of random numbers are generated to (approximately) calculate properties of probability distributions. In many cases the analytical form of these distributions is not known, so Monte Carlo methods are a great way to learn about them.

Transcription, the process by which DNA is transcribed into RNA, is key process in the central dogma of molecular biology. RNA polymerase (RNAP) is at the heart of this process.  This amazing machine glides along the DNA template, unzipping it internally, incorporating ribonucleotides at the front, and spitting RNA out the back. Sometimes, though, the polymerase pauses and then backtracks, pushing the RNA transcript back out the front, as shown in the figure below, taken from [Depken, et al., *Biophys. J.*, **96**, 2189-2193, 2009](https://dx.doi.org/10.1016%2Fj.bpj.2008.12.3918).

<center><img src="rnap_pause.png" width="400px"></center><br/>

To escape these backtracks, a cleavage enzyme called TFIIS cleaves the bit on RNA hanging out of the front, and the RNAP can then go about its merry way.

Researchers have long debated how these backtracks are governed. Single molecule experiments can provide some much needed insight. The groups of [Carlos Bustamante](http://bustamante.berkeley.edu), [Steve Block](https://blocklab.stanford.edu), and [Stephan Grill](https://www.mpi-cbg.de/research-groups/current-groups/stephan-grill/group-leader/), among others, have investigated the dynamics of RNAP in the absence of TFIIS.  They can measure many individual backtracks and get statistics about how long the backtracks last.

One hypothesis is that the backtracks simply consist of diffusive-like motion along the DNA stand. That is to say, the polymerase can move forward or backward along the strand with equal probability once it is paused. This is a one-dimensional **random walk**. So, if we want to test this hypothesis, we would want to know how much time we should expect the RNAP to be in a backtrack so that we could compare to experiment.

So, we seek the probability distribution of backtrack times, $P(t_{bt})$, where $t_{bt}$ is the time spent in the backtrack. We could solve this analytically, which requires some sophisticated mathematics. But, because we know how to draw random numbers, we can just compute this distribution directly using Monte Carlo simulation!

We start at $x = 0$ at time $t = 0$. We "flip a coin," or choose a random number to decide whether we step left or right. We do this again and again, keeping track of how many steps we take and what the $x$ position is. As soon as $x$ becomes positive, we have existed the backtrack. The total time for a backtrack is then $\tau n_\mathrm{steps}$, where $\tau$ is the time it takes to make a step. Depken, et al., report that $\tau \approx 0.5$ seconds.

**a)** Write a function, `backtrack_steps()`, that computes the number of steps it takes for a random walker (i.e., polymerase) starting at position $x = 0$ to get to position $x = +1$. It should return the number of steps to take the walk.

**b)** Generate 10,000 of these backtracks in order to get enough samples out of $P(t_\mathrm{bt})$. (If you are interested in a way to really speed up this calculation, ask me about [Numba](http://numba.pydata.org).)

**c)** Generate an ECDF of your samples and plot the ECDF with the $x$ axis on a logarithmic scale.

**d)** A probability distribution function that obeys a power law has the property

\begin{align}
P(t_\mathrm{bt}) \propto t_\mathrm{bt}^{-a}
\end{align}

in some part of the distribution, usually for large $t_\mathrm{bt}$. If this is the case, the cumulative distribution function is then

\begin{align}
\mathrm{CDF}(t_\mathrm{bt}) \equiv F(t_\mathrm{bt})= \int_{-\infty}^{t_\mathrm{bt}} \mathrm{d}t_\mathrm{bt}'\,P(t_\mathrm{bt}') = 1 - \frac{c}{t_\mathrm{bt}^{a+1}}, 
\end{align}

where $c$ is some constant defined by the functional form of $P(t_\mathrm{bt})$ for small $t_\mathrm{bt}$ and the normalization condition.  If $F$ is our cumulative histogram, we can check for power law behavior by plotting the complementary cumulative distribution (CCDF), $1 - F$, versus $t_\mathrm{bt}$.  If a power law is in play, the plot will be linear on a log-log scale with a slope of $-a+1$.

Plot the complementary cumulative distribution function from your samples on a log-log plot. If it is linear, then the time to exit a backtrack is a power law.

**e)**
By doing some mathematical heavy lifting, we know that, in the limit of large $t_{bt}$,

\begin{align}
P(t_{bt}) \propto t_{bt}^{-3/2},
\end{align}

so the plot you did in part (e) should have a slope of $-1/2$ on a log-log plot. Is this what you see?

**Notes:** The theory to derive the probability distribution is involved. See, e.g., [this](https://en.wikipedia.org/wiki/First-hitting-time_model). However, we were able to predict that we would see a great many short backtracks, and then see some very very long backtracks because of the power law distribution of backtrack times. We were able to do that just by doing a simple Monte Carlo simulation with "coin flips". There are many problems where the theory is *really* hard, and deriving the distribution is currently impossible, or the probability distribution has such an ugly expression that we can't really work with it. So, Monte Carlo methods are a powerful tool for generating predictions from simply-stated, but mathematically challenging, hypotheses.

Interestingly, many researchers thought (and maybe still do) there were two classes of backtracks: long and short. There may be, but the hypothesis that the backtrack is a random walk process is commensurate with seeing both very long and very short backtracks.

## Computing environment

In [4]:
%load_ext watermark
%watermark -v -p numpy,bokeh,jupyterlab

CPython 3.7.3
IPython 7.1.1

numpy 1.16.4
bokeh 1.2.0
jupyterlab 0.35.5
